In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# make deterministic
from mingpt.utils import set_seed
set_seed(44)

In [4]:
import os
import math
import time
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from data.othello import Othello, OthelloBoardState, permit_reverse
from mingpt.dataset import CharDataset
from mingpt.utils import sample
from mingpt.model import GPT, GPTConfig
from mingpt.trainer import Trainer, TrainerConfig

## Training

In [ ]:
# n_games=-1 means use as many simulated games as possible (from "data/othello_synthetic/")
othello = Othello(n_games=1000, data_root="othello_topleftbias80")
train_dataset = CharDataset(othello)
# original OthelloGPT params: n_layer=8, n_head=8, n_embd=512
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size, n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)

In [5]:
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    model = model.to(device)
else:
    print("NO GPU FOUND")

In [6]:
# setting up training
max_epochs = 40
experiment_name = "unbiased_e40"
t_start = time.strftime("_%Y%m%d_%H%M%S")
ckpt_path = f"./ckpts/{experiment_name}_{t_start}.ckpt"
tconf = TrainerConfig(
    max_epochs=max_epochs, 
    batch_size=512*4, # using 4 gpus
    learning_rate=5e-4,
    lr_decay=True, 
    warmup_tokens=len(train_dataset)*train_dataset.block_size*5, 
    final_tokens=len(train_dataset)*train_dataset.block_size*max_epochs,
    num_workers=0, 
    ckpt_path=ckpt_path, 
    # saved_epochs=[0, 1, 2, 3, 5, 10, 15, 20],
)
trainer = Trainer(model, train_dataset, None, tconf)
device = trainer.device
print(t_start)

_20230704_210935


In [ ]:
trainer.train()

## Validation

In [ ]:
def load_othello_model(ckpt):
    # original OthelloGPT params: n_layer=8, n_head=8, n_embd=512
    # vocab_size = 59, block_size = 61 for othello
    mconf = GPTConfig(61, 59, n_layer=8, n_head=8, n_embd=512)
    model = GPT(mconf)
    load_res = model.load_state_dict(torch.load(f"./ckpts/{ckpt}.ckpt"))
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        model = model.to(device)
        return model, device
    else:
        print("NO GPU FOUND")

In [29]:
# checks if model prediction is legal for each node in given game
# expects dataset has already been loaded and model is on GPU
def check_legal(model, device, train_dataset, game):
    total_nodes = 0
    success_nodes = 0

    len_whole_game = len(game)
    for len_partial_game in range(1, len_whole_game):
        total_nodes += 1
        context = game[:len_partial_game]
        x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None, ...].to(device)
        y = sample(model, x, 1, temperature=1.0)
        # taking top-1 prediction
        completion = [train_dataset.itos[int(i)] for i in y[0] if i != -1]
        try:
            OthelloBoardState().update(completion)
        except Exception:
            # print(completion)
            pass
        else:
            success_nodes += 1
    
    return total_nodes, success_nodes

In [30]:
# default data root is othello_synthetic
def validate_with_dataset(model, device, data_root=None, n_games=1000):
    # find to load in first n games, because the first ~1 million othello_synthetic games are test set for unbiased model
    val_games = Othello(data_root=data_root, n_games=n_games, test_split=1, deduplicate=False)
    char = CharDataset(val_games.val)

    total_nodes = 0
    success_nodes = 0

    def progress_report():
        return f"{success_nodes/total_nodes*100:.4f}% pass rate: {success_nodes}/{total_nodes} among all searched nodes"
    
    bar = tqdm(val_games.val[:n_games])
    for game in bar:
        tn, sn = check_legal(model, device, char, game)
        total_nodes += tn
        success_nodes += sn
        bar.set_description(progress_report())
    print(progress_report())

In [32]:
def validate_from_checkpoint(ckpt, data_root=None, n_games=1000):
    model, device = load_othello_model(ckpt)
    validate_with_dataset(model, device, data_root=data_root, n_games=n_games)

In [ ]:
validate_from_checkpoint("bias/TLcontrol", data_root="othello_topleftbias80")

In [ ]:
for c in ["bias/TLcontrol", "bias/TLbias50", "bias/TLbias80", "bias/TLbias95"]:
    for dr in ["synthetic", "TLbias50", "TLbias80", "TLbias95"]:
        print(f"======== ckpt: {c} | data: {dr} =========")
        validate_from_checkpoint(c, data_root=f"othello_{dr}", n_games=1000)

## Baselines

In [18]:
# eight directions
eights = [[-1, 0], [-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1]]
def check_adj(ob):
    total = 0
    occupied = ob.get_occupied()
    for i in range(64):
        r, c = i // 8, i % 8
        adj = False
        if occupied[i]:
            continue
        for dir in eights:
            test_r, test_c = r + dir[0], c + dir[1]
            if test_r not in range(8) or test_c not in range(8):
                continue
            if occupied[test_r * 8 + test_c]:
                adj = True
                break
        total += 1 if adj else 0
    return total

In [28]:
othello = Othello(data_root="othello_TLbias95", n_games=1000, test_split=0, deduplicate=False)
baselines = [
    0, # full random
    0, # no-repeats random
    0  # only adjacent
]
legal = 0
bar = tqdm(othello)
for seq in bar:
    ob = OthelloBoardState()
    for i, move in enumerate(seq):
        baselines[0] += 60
        baselines[1] += 60 - i
        baselines[2] += check_adj(ob)        
        legal += len(ob.get_valid_moves())
        ob.update([move])
    bar.set_description(desc=f"{legal}/{baselines[0]}, {legal/baselines[0]:.4f}")

for b in baselines:
    print(legal/b)

1912/21600, 0.0885:   0%|          | 3/1000 [00:00<00:38, 26.00it/s]

Loaded 100000 from 1 files
Using 1000 for training, 0 for validation


333621/3573780, 0.0934: 100%|██████████| 1000/1000 [00:38<00:00, 25.79it/s]

0.09335241676879942
0.18268010324937659
0.41459877417719254
